In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import logging


from graph import TransportGraph, ContactionTransportGraph
from ttf import TTF
from forward_search import FCH
from dijkstra import Dijkstra

# Build transport graph

In [2]:
CITY = 'kuopio'

## Build Transport Graph

In [3]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')

In [4]:
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

In [5]:
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

# Convert ATF to TTF

In [6]:
tg_ttf = deepcopy(tg)
for node1, out in tg_ttf.graph.items():
    for node2, f in out.items():
        tg_ttf.graph[node1][node2] = TTF(f)

## Graph statistics

In [6]:
tg.edges_cnt, tg.nodes_cnt, tg.timetable_stats

(8891,
 549,
 {'min_size': 0,
  'mean_size': 3.2843324710381285,
  'std_size': 15.760627889293113,
  'max_size': 306})

### Save TG with ATF and TG with TTF

In [ ]:
pickle.dump(tg, open(F'{CITY}.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttf, open(F'{CITY}_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Build CH graph

In [7]:
%%time
cProfile.run('ch_tg = tg.contraction_hierarchy()')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [02:14<00:00,  4.07it/s]

         186247595 function calls (183514399 primitive calls) in 145.656 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:100(acquire)
     10/3    0.000    0.000    0.080    0.027 <frozen importlib._bootstrap>:1022(_find_and_load)
     14/7    0.000    0.000    0.006    0.001 <frozen importlib._bootstrap>:1053(_handle_fromlist)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:125(release)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:165(__init__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:169(__enter__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:173(__exit__)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:179(_get_module_lock)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstra

## Graph statistics

In [8]:
ch_tg.edges_cnt, ch_tg.nodes_cnt, ch_tg.timetable_stats

(18736,
 549,
 {'min_size': 0,
  'mean_size': 42.07723099914603,
  'std_size': 54.05542987204912,
  'max_size': 327})

### Calculate Geometrical Containers for speed up future search

In [9]:
ch_tg.geometrical_container()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:00<00:00, 1979.84it/s]


# Convert ATF to TTF in CH-graph

In [ ]:
ch_tg_ttf = deepcopy(ch_tg)
for node1, out in ch_tg_ttf.graph.items():
    for node2, f in out.items():
        ch_tg_ttf.graph[node1][node2] = TTF(f)

## Save graph

In [ ]:
pickle.dump(ch_tg, open(F'{CITY}_ch.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg, open(F'{CITY}_ch_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Calulate TTN for TG AND CH-graph

In [64]:
pickle.dump(ch_tg, open(F'graph/ch_{CITY}_fractional.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [61]:
ch_tg.position_in_edge = defaultdict(dict)
ch_tg.nodes_schedule = defaultdict(dict)

In [62]:
ch_tg.m_arr_fractional = {}
ch_tg.pointers = {}
ch_tg.reachable_nodes = {}

In [65]:
tg.optimize_binary_search()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:00<00:00, 1442.61it/s]


In [66]:
tg.fractional_cascading_precomputation()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:00<00:00, 7190.28it/s]


In [67]:
ch_tg.optimize_binary_search()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:09<00:00, 58.02it/s]


In [68]:
ch_tg.fractional_cascading_precomputation()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:05<00:00, 101.54it/s]


# Compare solutions

In [36]:
N = 1000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)]
             })

algorithms = ['fch', 'fch_fractional', 'fch_binary_duration', 'dijkstra', 'dijkstra_binary_duration', 'dijkstra_fractional']
duration = {x: [] for x in algorithms}
arrival = {x: 0 for x in algorithms}
for index, row in test_data.iterrows():
    random.shuffle(algorithms)
    for algorithm in algorithms:
        if algorithm == 'fch':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, 
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_fractional':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, 
                                             fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_binary_duration':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True,
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
            
        elif algorithm == 'dijkstra':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_fractional':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_binary_duration':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
    arr = arrival[algorithms[0]]        
    for i in range(1, len(algorithms)):
        assert arr == arrival[algorithms[i]]  

/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_6996/2178912940.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)],
/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_6996/2178912940.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)]


In [38]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

fch_binary_duration 10.437429537767757 10.0 7.638999602433691
fch 15.110484780157835 13.0 21.86097232601847
dijkstra_binary_duration 15.816234498308907 16.0 9.03014157402458
fch_fractional 14.638105975197295 12.0 44.66875242366695
dijkstra_fractional 16.931228861330325 17.0 13.590599782047326
dijkstra 17.24013528748591 17.0 10.58736410415351


# Results

In [ ]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))